# CREWAI

# Prerequisiti (consigliati)

* **Python 3.10+** installato (`python --version`).
* **pip** o **uv/pipx** per installare pacchetti.
* Facoltativo ma utile: **Git** per versionare il progetto.
* Un editor (VS Code, PyCharm, ecc.).

> Suggerimento: lavora in un **virtual environment** per isolare le dipendenze.

```bash
# (Opzionale) crea e attiva un venv
python -m venv venv
# Windows
venv\Scripts\activate
# macOS/Linux
source venv/bin/activate
```

---

# Step 1 — Crea un nuovo progetto CrewAI

## 1) Installa il CLI (se non lo hai già)

```bash
pip install crewai
```

> Se il comando `crewai` non viene trovato, chiudi e riapri il terminale (oppure assicurati che lo `Scripts`/`bin` del tuo venv sia nel PATH).

## 2) Genera il progetto con il comando CLI

```bash
crewai create crew research_crew
cd research_crew
```

**Cosa fa questo comando**

* Crea la cartella `research_crew/` con una struttura “pronta all’uso”.
* Genera file Python e YAML di base per **agenti** e **task**.
* Prepara un **entrypoint** (`main.py`) per avviare la crew.

## 3) Prepara le variabili d’ambiente

Apri il file **`.env`** creato nella root del progetto e inserisci le tue chiavi (non committarle su Git):

Esempi (scegline uno in base al provider):

```dotenv
# OpenAI (esempio)
OPENAI_API_KEY=sk-...

# LM Studio / endpoint OpenAI-compatible (esempio)
OPENAI_BASE_URL=http://localhost:1234/v1
OPENAI_API_KEY=lm-studio
```

> Le chiavi **non** vanno messe nei file YAML: mantienile sempre nel `.env`.


# Step 2 — Esplora la struttura generata

Struttura tipica generata dal CLI:

```
research_crew/
├── .gitignore
├── pyproject.toml
├── README.md
├── .env
└── src/
    └── research_crew/
        ├── __init__.py
        ├── main.py
        ├── crew.py
        ├── tools/
        │   ├── custom_tool.py
        │   └── __init__.py
        └── config/
            ├── agents.yaml
            └── tasks.yaml
```

## Cosa contiene e a cosa serve

* **.gitignore**
  Elenca ciò che Git deve ignorare (ad es. `venv/`, file temporanei, cache). Evita di versionare segreti e build.

* **pyproject.toml**
  Definisce il progetto Python (nome, dipendenze, metadati).

  * Dove aggiungere librerie extra (`langchain`, `python-dotenv`, ecc.).
  * Abilita l’installazione in `editable` (`pip install -e .`).

* **README.md**
  Istruzioni d’uso del progetto. Aggiornalo con comandi e note per chi collaborerà con te.

* **.env**
  File locale con le **chiavi** e le **config** sensibili (API keys, URL modello). Non committarlo.

* **src/research\_crew/**init**.py**
  Rende `research_crew` un **package** Python (importabile). Di solito resta vuoto.

* **src/research\_crew/main.py**
  **Entry point** del progetto. Di norma:

  * importa la Crew da `crew.py`;
  * raccoglie eventuali **inputs** (es. `{"topic": "AI LLMs"}`) da riga di comando o dal codice;
  * esegue `crew().kickoff(...)` o comandi simili.
    Qui puoi aggiungere parsing degli argomenti o varianti “train/test”.

* **src/research\_crew/crew\.py**
  Il cuore dell’orchestrazione:

  * Carica **agenti** da `config/agents.yaml`.
  * Carica **task** da `config/tasks.yaml`.
  * Assembla la **Crew** (pipeline/sequenza/agenti) e il **Process** (sincrono, parallelo, gerarchico).
  * Registra gli eventuali **tools** (vedi sotto).

* **src/research\_crew/tools/**
  Dove mettere i tuoi **strumenti personalizzati** (funzioni che gli agenti possono invocare: scraping, chiamate DB/API, calcoli…).

  * `custom_tool.py`: esempio di tool; duplicalo per crearne altri.
  * `__init__.py`: esporta i tools per un import pulito in `crew.py`.






# Step 3: Configura i tuoi Agenti

Adesso arriva la parte interessante: definire gli **agenti AI**.
In CrewAI ogni agente ha un **ruolo**, un **obiettivo** e un **background** che ne guidano il comportamento. Puoi immaginarli come **personaggi di una recita**, ognuno con la propria specializzazione e il proprio approccio.

Per il nostro esempio di *research crew* definiamo due figure:

* un **researcher**, esperto nella raccolta e organizzazione delle informazioni;
* un **analyst**, capace di interpretare i dati e trasformarli in report chiari e utili.

Questa configurazione va scritta nel file **`agents.yaml`**:

```yaml
# src/research_crew/config/agents.yaml

researcher:
  role: >
    Senior Research Specialist for {topic}
  goal: >
    Find comprehensive and accurate information about {topic}
    with a focus on recent developments and key insights
  backstory: >
    You are an experienced research specialist with a talent for
    finding relevant information from various sources. You excel at
    organizing information in a clear and structured manner, making
    complex topics accessible to others.
  llm: provider/model-id  # es. openai/gpt-4o, google/gemini-2.0-flash, anthropic/claude...

analyst:
  role: >
    Data Analyst and Report Writer for {topic}
  goal: >
    Analyze research findings and create a comprehensive, well-structured
    report that presents insights in a clear and engaging way
  backstory: >
    You are a skilled analyst with a background in data interpretation
    and technical writing. You have a talent for identifying patterns
    and extracting meaningful insights from research data, then
    communicating those insights effectively through well-crafted reports.
  llm: provider/model-id  # es. openai/gpt-4o, google/gemini-2.0-flash, anthropic/claude...
```

 Nota: ruoli, obiettivi e background **non sono semplici descrizioni**, ma influiscono direttamente sul modo in cui l’agente affronterà i compiti. Scriverli bene significa ottenere comportamenti più mirati e specializzati.

---

# Step 4: Definisci i tuoi Task

Dopo aver creato gli agenti, è il momento di assegnare loro i **compiti concreti**.
I task sono le attività operative che gli agenti dovranno svolgere, con istruzioni precise e output attesi.

Per il nostro team di ricerca, definiamo due task principali:

1. un **research task** che raccoglie informazioni complete,
2. un **analysis task** che produce un report strutturato con analisi e raccomandazioni.

La configurazione si scrive in **`tasks.yaml`**:

```yaml
# src/research_crew/config/tasks.yaml

research_task:
  description: >
    Conduct thorough research on {topic}. Focus on:
    1. Key concepts and definitions
    2. Historical development and recent trends
    3. Major challenges and opportunities
    4. Notable applications or case studies
    5. Future outlook and potential developments

    Make sure to organize your findings in a structured format with clear sections.
  expected_output: >
    A comprehensive research document with well-organized sections covering
    all the requested aspects of {topic}. Include specific facts, figures,
    and examples where relevant.
  agent: researcher

analysis_task:
  description: >
    Analyze the research findings and create a comprehensive report on {topic}.
    Your report should:
    1. Begin with an executive summary
    2. Include all key information from the research
    3. Provide insightful analysis of trends and patterns
    4. Offer recommendations or future considerations
    5. Be formatted in a professional, easy-to-read style with clear headings
  expected_output: >
    A polished, professional report on {topic} that presents the research
    findings with added analysis and insights. The report should be well-structured
    with an executive summary, main sections, and conclusion.
  agent: analyst
  context:
    - research_task
  output_file: output/report.md
```

 Nota importante:
Nel secondo task è presente il campo **`context`**, che permette all’analyst di accedere direttamente all’output del **research\_task**.
In questo modo si crea un **flusso di lavoro naturale**, dove l’analista utilizza i risultati del ricercatore, esattamente come avverrebbe in un team umano.

---


# Step 5: Configura la tua Crew

Ora è il momento di **mettere insieme tutti i pezzi**: gli agenti e i task che abbiamo definito vanno orchestrati dentro una **crew**.
La crew è il contenitore che coordina come gli agenti lavorano insieme per portare a termine i compiti, un po’ come il “regista” che organizza gli attori in una rappresentazione teatrale.

Il file da modificare è **`crew.py`**, che si trova in `src/research_crew/`.

---

## Codice di esempio

Ecco la versione di base che il CLI ha preparato e che andiamo a completare:

```python
# src/research_crew/crew.py
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List

@CrewBase
class ResearchCrew():
    """Research crew for comprehensive topic analysis and reporting"""

    agents: List[BaseAgent]
    tasks: List[Task]

    @agent
    def researcher(self) -> Agent:
        return Agent(
            config=self.agents_config['researcher'], # type: ignore[index]
            verbose=True,
            # tools=[SerperDevTool()] possibilità di usare tool al momento disattivata
        )

    @agent
    def analyst(self) -> Agent:
        return Agent(
            config=self.agents_config['analyst'], # type: ignore[index]
            verbose=True
        )

    @task
    def research_task(self) -> Task:
        return Task(
            config=self.tasks_config['research_task'] # type: ignore[index]
        )

    @task
    def analysis_task(self) -> Task:
        return Task(
            config=self.tasks_config['analysis_task'], # type: ignore[index]
            output_file='output/report.md'
        )

    @crew
    def crew(self) -> Crew:
        """Creates the research crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )
```

---

## Spiegazione passo per passo

### 1. Decoratore `@CrewBase`

La classe `ResearchCrew` è decorata con `@CrewBase`.
Questo permette a CrewAI di riconoscerla come **definizione di crew**, caricando automaticamente agenti e task dalle configurazioni YAML (`agents.yaml` e `tasks.yaml`).

---

### 2. Definizione degli agenti

* **Metodo `researcher`**
  Qui creiamo l’agente “ricercatore”, prendendo la sua configurazione dal file `agents.yaml` (`self.agents_config['researcher']`).

  * `verbose=True` significa che vedremo a schermo log dettagliati di cosa fa.
  * `tools=[SerperDevTool()]`: aggiungiamo un tool esterno (in questo caso Serper, per fare ricerche web).
    → Questo arricchisce l’agente permettendogli di ottenere informazioni aggiornate dalla rete.

* **Metodo `analyst`**
  Crea l’agente “analista”, sempre basandosi sulla config YAML.
  Non ha tool extra, perché il suo lavoro è **analizzare i dati già raccolti** e non cercare nuove informazioni.

---

### 3. Definizione dei task

* **Metodo `research_task`**
  Collega la configurazione del task di ricerca (`tasks.yaml`) con l’agente `researcher`.
  Non fa altro: la logica è già descritta in YAML.

* **Metodo `analysis_task`**
  Collega la configurazione del task di analisi.
  In più, aggiunge un parametro extra:

  ```python
  output_file='output/report.md'
  ```

  Questo significa che il risultato finale del task (il report generato) verrà **salvato come file Markdown** in `output/report.md`.
  Utile se vuoi leggere, condividere o versionare l’output.

---

### 4. Definizione della crew

* **Metodo `crew`**
  È qui che uniamo tutto.

  ```python
  return Crew(
      agents=self.agents,
      tasks=self.tasks,
      process=Process.sequential,
      verbose=True,
  )
  ```

  * `agents=self.agents`: la crew utilizza tutti gli agenti che abbiamo definito sopra.
  * `tasks=self.tasks`: stesso discorso per i task.
  * `process=Process.sequential`: significa che i task vengono eseguiti **in sequenza**, uno dopo l’altro.
    In questo caso:

    1. prima il `research_task` (ricerca),
    2. poi l’`analysis_task` (analisi), che userà i risultati del primo.
  * `verbose=True`: di nuovo, log dettagliati.

---

## Cosa abbiamo ottenuto

Con pochissimo codice:

* Abbiamo **collegato agenti e task** alle configurazioni YAML.
* Abbiamo arricchito il **researcher** con un tool (Serper per le ricerche).
* Abbiamo deciso il **flusso di esecuzione** (prima ricerca, poi analisi).
* Abbiamo configurato l’output del report in un file `.md`.

In pratica abbiamo costruito un **sistema collaborativo di agenti AI** che si comportano come un team umano:

1. Uno raccoglie i dati (ricercatore).
2. L’altro li interpreta e produce un documento professionale (analista).

---


# Step 6: Main script per avviare la crew

Il main script serve a:

1. preparare l’ambiente (cartella `output/`),
2. passare gli **inputs** (qui: il `{topic}`),
3. avviare la crew e stampare/salvare il risultato.

Usa questo file in `src/research_crew/main.py`:

```python
#!/usr/bin/env python
# src/research_crew/main.py
import os
from research_crew.crew import ResearchCrew

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

def run():
    """
    Run the research crew.
    """
    inputs = {
        'topic': 'Artificial Intelligence in Healthcare'
    }

    # Create and run the crew
    result = ResearchCrew().crew().kickoff(inputs=inputs)

    # Print the result
    print("\n\n=== FINAL REPORT ===\n\n")
    print(result.raw)

    print("\n\nReport has been saved to output/report.md")

if __name__ == "__main__":
    run()
```

Cosa fa:

* **Crea la cartella** `output/` se non esiste (evita errori di scrittura).
* Definisce l’**input** richiesto nei YAML (`{topic}`).
* Avvia la crew con `kickoff(inputs=...)`.
* **Stampa** il risultato completo a video e conferma il salvataggio su `output/report.md` (configurato nel task di analisi).

---





# Step 8: Installa le dipendenze

Per far funzionare il progetto, dobbiamo assicurarci che tutte le librerie necessarie siano installate. CrewAI semplifica questo con un solo comando:

```bash
crewai install
```

### Cosa fa questo comando:

* **Legge la configurazione del progetto** (`pyproject.toml`) per capire quali pacchetti servono.
* **Crea automaticamente un virtual environment**, se non ne hai già uno attivo.
* **Installa tutte le dipendenze** richieste dal progetto (CrewAI, tool, librerie esterne).

 Questo step è importante per garantire che il tuo progetto sia replicabile su qualsiasi macchina senza dover ricordare manualmente ogni pacchetto.

---

# Step 9: Esegui la tua crew

Ed eccoci al momento più atteso: far “partire” la tua squadra di agenti AI.
Per avviarla usa:

```bash
crewai run
```

### Cosa succede durante l’esecuzione:

1. Il **researcher** parte per primo:

   * raccoglie informazioni sul `{topic}` definito in `main.py`;
   * utilizza eventuali tool abilitati (es. `SerperDevTool` per cercare sul web).

2. Una volta concluso, entra in azione l’**analyst**:

   * riceve l’output del task di ricerca;
   * lo analizza, ne individua tendenze e pattern;
   * produce un **report strutturato e leggibile**.

3. Durante l’esecuzione vedrai sul terminale:

   * i **ragionamenti** degli agenti (prompt e step intermedi);
   * le azioni compiute (ad esempio chiamate a tool esterni);
   * i risultati parziali e infine l’output finale.

 È proprio questa la forza di CrewAI: gestire in automatico il coordinamento tra più agenti, senza dover scrivere tu stesso tutta la logica di passaggio dati.

---

# Step 10: Controlla l’output

Una volta terminata l’esecuzione, troverai il report finale nella cartella:

```
output/report.md
```

Il contenuto tipico del file sarà:

* **Executive summary** (sintesi iniziale).
* **Informazioni dettagliate** sul tema scelto.
* **Analisi e approfondimenti** basati sui dati raccolti.
* **Raccomandazioni e prospettive future**.

 Apri il file con un editor di testo o direttamente in VS Code: avrai un documento leggibile e pronto da condividere o utilizzare in altri contesti.

---

# Altri comandi utili del CLI

CrewAI fornisce vari comandi extra che ti aiutano a gestire il ciclo di vita della tua crew:

```bash
# Mostra tutti i comandi disponibili
crewai --help

# Esegue la crew (equivalente a python -m research_crew.main)
crewai run

# Avvia i test del progetto
crewai test

# Resetta le memorie della crew
crewai reset-memories

# Riavvia dall'esecuzione di un task specifico
crewai replay -t <task_id>
```

Questi comandi sono utili soprattutto se intendi sviluppare progetti più complessi, testare singoli agenti o simulare scenari particolari.

---



# FLOWS

# 1) Capire cosa sono i Flows (e perché usarli)

* **Idea chiave**: i *Flows* combinano la collaborazione tra agenti (crews) con la **precisione del controllo procedurale** (quando, come e in che ordine avvengono le cose).
* **Quando servono**: quando vuoi **decidere tu** l’ordine dei passi, reagire a **eventi**, mantenere **stato** condiviso e **integrare** codice normale, chiamate LLM dirette e processi di crew.

# 2) Obiettivo della guida che costruirai

* **Scopo**: generare una **learning guide** completa su un qualsiasi argomento.
* **Come**: combinando input utente, una fase di **pianificazione** (LLM diretto), una fase di **produzione contenuti** (crew multi-agente) e una **orchestrazione** che lega tutto in modo strutturato ed event-driven.

# 3) Scegliere i “pattern” di interazione AI

* **Crews**: per compiti collaborativi e articolati (es. ricerca approfondita + stesura report).
* **LLM diretto**: per operazioni più semplici e mirate (es. generare l’indice/outline della guida).
* **Codice regolare**: per la logica di controllo (validazioni, trasformazioni, unioni di testi, formattazione finale).

# 4) Pensare in ottica event-driven

* **Eventi**: ogni step “ascolta” il completamento del precedente (es. *“quando c’è il topic, crea l’outline; quando c’è l’outline, avvia la crew…”*).
* **Reazioni**: definisci cosa deve accadere **dopo** ogni evento (prossimo step, trasformazione dati, salvataggio intermedio).
* **Vantaggio**: il flusso è **prevedibile** e **tracciabile**, perché ogni passaggio scatta al momento giusto.

# 5) Mantenere lo “stato” lungo il flusso

* **Stato condiviso**: una struttura dati che conserva tutto ciò che serve:

  * `topic` (input utente),
  * `outline/plan` (ottenuto da LLM diretto),
  * `ricerca` e `report` (provenienti dalla crew),
  * `learning_guide` finale (testo composto).
* **Perché è utile**: permette di **riusare** e **trasformare** i risultati tra gli step senza perderli.

# 6) Disegnare il percorso di esecuzione

* **Sequenziale**: per la guida base, di solito:

  1. ricevi `topic`,
  2. genera **outline**,
  3. esegui **crew** (ricerca → analisi),
  4. **componi** la guida finale.
* **Condizionale**: puoi prevedere **rami diversi** (es. outline corto → percorso breve; outline ricco → percorso completo).
* **Parallelo**: alcuni step possono partire **in parallelo** dopo un evento comune (es. creare anche una “lista risorse” mentre si compone la guida).

# 7) Integrare sistemi esterni (senza dettagli tecnici)

* **Cosa significa**: collegare il flusso a **database**, **API**, o **interfacce utente**.
* **Esempi di integrazione**: salvare versioni intermedie, inviare notifiche, leggere dati aggiuntivi.
  (Resta concettuale: l’idea è che i Flows lo rendono naturale.)

# 8) Orchestrare il passaggio dati

* **Input/Output chiari**: ogni step produce un output che diventa input del successivo.
* **Trasformazioni**: normalizza, pulisci, struttura i dati tra uno step e l’altro (es. dai “punti ricerca” a “sezioni del report”).

# 9) Perché i Flows sono potenti (riassunto pratico)

* **Mix di pattern**: crews + LLM diretto + codice.
* **Eventi**: sistemi che **reagiscono** a cambi di stato e completamenti.
* **Stato persistente**: dati condivisi e trasformati lungo la pipeline.
* **Integrazione facile**: con DB, API, UI.
* **Percorsi complessi**: rami condizionali, parallelismi, workflow dinamici.

# 10) Strade di evoluzione (solo esempi citati)

* **Assistenti interattivi** con sottosistemi specializzati.
* **Pipeline dati complesse** con trasformazioni potenziate dall’AI.
* **Agenti autonomi** integrati con servizi esterni e API.
* **Processi multi-stadio** con momenti di **human-in-the-loop**.

---

## In una frase

Con i Flows costruisci **workflow AI strutturati ed event-driven**: decidi tu *quando* e *come* far cooperare LLM, crew di agenti e codice, mantenendo **stato** e **controllo preciso** su tutto il processo di creazione della tua learning guide.


---

# Step 1: Creare un nuovo progetto Flow

Per creare un progetto di tipo *Flow* con CrewAI si utilizza il **CLI**.
Da terminale, esegui:

```bash
crewai create flow guide_creator_flow
cd guide_creator_flow
```

### Cosa fa questo comando:

1. **Genera una cartella** `guide_creator_flow/` con tutto il necessario per iniziare.
2. Crea i file di configurazione di base (come `.gitignore`, `pyproject.toml`, `.env`).
3. Prepara un **template di flow** pronto da modificare.
4. Inserisce già un esempio di **crew specializzata** (qui: `poem_crew`) che ti aiuterà a capire come integrare più squadre di agenti.

 L’idea è che tu non debba reinventare la struttura da zero: il CLI crea uno “scheletro” ordinato e conforme alle best practice.

---

# Step 2: Capire la struttura del progetto

Il progetto appena creato apparirà così:

```
guide_creator_flow/
├── .gitignore
├── pyproject.toml
├── README.md
├── .env
├── main.py
├── crews/
│   └── poem_crew/
│       ├── config/
│       │   ├── agents.yaml
│       │   └── tasks.yaml
│       └── poem_crew.py
└── tools/
    └── custom_tool.py
```

Vediamo in dettaglio i componenti principali:

### File principali

* **`.gitignore`**
  Contiene la lista di file/cartelle da escludere dal versionamento Git (es. `venv/`, `.env`, cache).

* **`pyproject.toml`**
  File di configurazione del progetto Python.

  * Definisce le dipendenze.
  * Permette di installare il progetto in modalità `editable` (`pip install -e .`).

* **`README.md`**
  Documentazione di base: spiega lo scopo del progetto e come usarlo. Da personalizzare.

* **`.env`**
  File dedicato alle **variabili d’ambiente** (es. API key per provider LLM o tool esterni).
  È il posto giusto per mettere segreti senza rischiare di committarli.

* **`main.py`**
  È il cuore del flow: qui definirai la logica principale che coordina input utente, crews e tools.

  * Può lanciare la crew `poem_crew`.
  * Sarà adattato in seguito per gestire il **guide creator flow**.

---

### Directory `crews/`

Contiene i singoli team di agenti AI. Ogni crew è autonoma e ha:

* **`config/agents.yaml`** → definisce i ruoli, obiettivi e backstory degli agenti.
* **`config/tasks.yaml`** → assegna i compiti (descrizione, output attesi, agente responsabile).
* **`poem_crew.py`** → file Python che unisce agenti e task per creare una crew funzionante.

 Nel tuo caso, `poem_crew` è un esempio. Lo userai come modello per costruire la tua **guide\_creator\_crew** in seguito.

---

### Directory `tools/`

* **`custom_tool.py`**
  Qui puoi creare strumenti personalizzati che i tuoi agenti possono usare (es. chiamate API, scraping, calcoli specifici).
  È il posto per integrare funzioni esterne che arricchiscono le capacità degli agenti.

---

## In sintesi

* **Flow logic** → in `main.py`
* **Crew specializzate** → in `crews/` con agenti e task definiti in YAML + Python
* **Tools personalizzati** → in `tools/`

Questa separazione è utile perché:

* puoi cambiare facilmente il comportamento di un agente modificando solo i file YAML;
* puoi aggiungere nuove crew senza toccare il resto del progetto;
* puoi mantenere ordinata la logica, distinguendo configurazioni, orchestrazione e strumenti.

---




#  Step 3: Aggiungere una Content Writer Crew

1. Dal terminale, esegui il comando:

   ```bash
   crewai flow add-crew content-crew
   ```

   Questo:

   * crea la cartella `content_crew/` dentro `crews/`,
   * aggiunge i file necessari (`agents.yaml`, `tasks.yaml`, `content_crew.py`),
   * prepara la struttura per inserire agenti e task dedicati alla scrittura dei contenuti.

 L’idea è che il nostro **flow principale** possa orchestrare più crew specializzate. La **content crew** è un piccolo “team editoriale” che si occuperà di redigere e revisionare le sezioni della guida.

---

#  Step 4: Configurare la Content Writer Crew

Ora personalizziamo i file generati. Qui entrano in gioco i **ruoli degli agenti** e i **task** da svolgere.

---

## 1. Definire gli agenti (agents.yaml)

File:
`src/guide_creator_flow/crews/content_crew/config/agents.yaml`

Abbiamo due figure:

* **Content Writer**: scrive sezioni educative chiare e coinvolgenti.
* **Content Reviewer**: controlla e migliora ciò che il writer ha prodotto.

```yaml
content_writer:
  role: >
    Educational Content Writer
  goal: >
    Create engaging, informative content that thoroughly explains the assigned topic
    and provides valuable insights to the reader
  backstory: >
    You are a talented educational writer with expertise in creating clear, engaging
    content. You have a gift for explaining complex concepts in accessible language
    and organizing information in a way that helps readers build their understanding.
  llm: provider/model-id  # es. openai/gpt-4o, google/gemini-2.0-flash, anthropic/claude...

content_reviewer:
  role: >
    Educational Content Reviewer and Editor
  goal: >
    Ensure content is accurate, comprehensive, well-structured, and maintains
    consistency with previously written sections
  backstory: >
    You are a meticulous editor with years of experience reviewing educational
    content. You have an eye for detail, clarity, and coherence. You excel at
    improving content while maintaining the original author's voice and ensuring
    consistent quality across multiple sections.
  llm: provider/model-id
```

 Qui la cosa importante è che i due agenti hanno **ruoli complementari**: uno crea, l’altro rifinisce. Insieme, coprono l’intero ciclo di produzione editoriale.

---

## 2. Definire i task (tasks.yaml)

File:
`src/guide_creator_flow/crews/content_crew/config/tasks.yaml`

I task stabiliscono **cosa fanno gli agenti** e in quale ordine.

* **Write Section Task** → affidato al *content\_writer*: scrive una sezione completa di circa 500-800 parole, con esempi, applicazioni pratiche e formattazione Markdown.
* **Review Section Task** → affidato al *content\_reviewer*: prende in input la sezione scritta e la migliora, garantendo qualità, consistenza e chiarezza.

```yaml
write_section_task:
  description: >
    Write a comprehensive section on the topic: "{section_title}"
    ...
  expected_output: >
    A well-structured, comprehensive section in Markdown format ...
  agent: content_writer

review_section_task:
  description: >
    Review and improve the following section on "{section_title}":
    ...
  expected_output: >
    An improved, polished version of the section ...
  agent: content_reviewer
  context:
    - write_section_task
```

 Nota fondamentale: il **campo `context`**.
Serve per passare l’output del task di scrittura direttamente al task di revisione. In pratica, il reviewer lavora sempre sul testo prodotto dal writer. Questo ricrea il workflow naturale: prima scrittura, poi revisione.

---

## 3. Implementazione della crew (content\_crew\.py)

File:
`src/guide_creator_flow/crews/content_crew/content_crew.py`

Qui colleghiamo agenti e task:

```python
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List

@CrewBase
class ContentCrew():
    """Content writing crew"""

    agents: List[BaseAgent]
    tasks: List[Task]

    @agent
    def content_writer(self) -> Agent:
        return Agent(
            config=self.agents_config['content_writer'],
            verbose=True
        )

    @agent
    def content_reviewer(self) -> Agent:
        return Agent(
            config=self.agents_config['content_reviewer'],
            verbose=True
        )

    @task
    def write_section_task(self) -> Task:
        return Task(
            config=self.tasks_config['write_section_task']
        )

    @task
    def review_section_task(self) -> Task:
        return Task(
            config=self.tasks_config['review_section_task'],
            context=[self.write_section_task()]
        )

    @crew
    def crew(self) -> Crew:
        """Creates the content writing crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )
```

 Cosa succede qui:

* `@agent`: dichiara i due agenti (writer e reviewer).
* `@task`: lega i task YAML alle funzioni della crew.
* `context=[...]`: il reviewer riceve come input il risultato del writer.
* `@crew`: mette tutto insieme in un processo **sequenziale**: prima scrittura → poi revisione.

---
 Risultato

Abbiamo creato un **mini-team editoriale AI**:

* **Writer** → genera una bozza completa.
* **Reviewer** → la migliora e garantisce coerenza.

Questo meccanismo riflette il flusso editoriale umano ed è pronto per essere orchestrato dal flow principale (`main.py`).

---


# MAIN.PY

## CREIAMO IL NOSTRO FLOW

```python
#!/usr/bin/env python
import json
import os
from typing import List, Dict
from pydantic import BaseModel, Field
from crewai import LLM
from crewai.flow.flow import Flow, listen, start
from guide_creator_flow.crews.content_crew.content_crew import ContentCrew

# Define our models for structured data
class Section(BaseModel):
    title: str = Field(description="Title of the section")
    description: str = Field(description="Brief description of what the section should cover")

class GuideOutline(BaseModel):
    title: str = Field(description="Title of the guide")
    introduction: str = Field(description="Introduction to the topic")
    target_audience: str = Field(description="Description of the target audience")
    sections: List[Section] = Field(description="List of sections in the guide")
    conclusion: str = Field(description="Conclusion or summary of the guide")

# Define our flow state
class GuideCreatorState(BaseModel):
    topic: str = ""
    audience_level: str = ""
    guide_outline: GuideOutline = None
    sections_content: Dict[str, str] = {}

class GuideCreatorFlow(Flow[GuideCreatorState]):
    """Flow for creating a comprehensive guide on any topic"""

    @start()
    def get_user_input(self):
        """Get input from the user about the guide topic and audience"""
        print("\n=== Create Your Comprehensive Guide ===\n")

        # Get user input
        self.state.topic = input("What topic would you like to create a guide for? ")

        # Get audience level with validation
        while True:
            audience = input("Who is your target audience? (beginner/intermediate/advanced) ").lower()
            if audience in ["beginner", "intermediate", "advanced"]:
                self.state.audience_level = audience
                break
            print("Please enter 'beginner', 'intermediate', or 'advanced'")

        print(f"\nCreating a guide on {self.state.topic} for {self.state.audience_level} audience...\n")
        return self.state

    @listen(get_user_input)
    def create_guide_outline(self, state):
        """Create a structured outline for the guide using a direct LLM call"""
        print("Creating guide outline...")

        # Initialize the LLM
        llm = LLM(model="azure/gpt-4o", response_format=GuideOutline)

        # Create the messages for the outline
        messages = [
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": f"""
            Create a detailed outline for a comprehensive guide on "{state.topic}" for {state.audience_level} level learners.

            The outline should include:
            1. A compelling title for the guide
            2. An introduction to the topic
            3. 4-6 main sections that cover the most important aspects of the topic
            4. A conclusion or summary

            For each section, provide a clear title and a brief description of what it should cover.
            """}
        ]

        # Make the LLM call with JSON response format
        response = llm.call(messages=messages)

        # Parse the JSON response
        outline_dict = json.loads(response)
        self.state.guide_outline = GuideOutline(**outline_dict)

        # Ensure output directory exists before saving
        os.makedirs("output", exist_ok=True)

        # Save the outline to a file
        with open("output/guide_outline.json", "w") as f:
            json.dump(outline_dict, f, indent=2)

        print(f"Guide outline created with {len(self.state.guide_outline.sections)} sections")
        return self.state.guide_outline

    @listen(create_guide_outline)
    def write_and_compile_guide(self, outline):
        """Write all sections and compile the guide"""
        print("Writing guide sections and compiling...")
        completed_sections = []

        # Process sections one by one to maintain context flow
        for section in outline.sections:
            print(f"Processing section: {section.title}")

            # Build context from previous sections
            previous_sections_text = ""
            if completed_sections:
                previous_sections_text = "# Previously Written Sections\n\n"
                for title in completed_sections:
                    previous_sections_text += f"## {title}\n\n"
                    previous_sections_text += self.state.sections_content.get(title, "") + "\n\n"
            else:
                previous_sections_text = "No previous sections written yet."

            # Run the content crew for this section
            result = ContentCrew().crew().kickoff(inputs={
                "section_title": section.title,
                "section_description": section.description,
                "audience_level": self.state.audience_level,
                "previous_sections": previous_sections_text,
                "draft_content": ""
            })

            # Store the content
            self.state.sections_content[section.title] = result.raw
            completed_sections.append(section.title)
            print(f"Section completed: {section.title}")

        # Compile the final guide
        guide_content = f"# {outline.title}\n\n"
        guide_content += f"## Introduction\n\n{outline.introduction}\n\n"

        # Add each section in order
        for section in outline.sections:
            section_content = self.state.sections_content.get(section.title, "")
            guide_content += f"\n\n{section_content}\n\n"

        # Add conclusion
        guide_content += f"## Conclusion\n\n{outline.conclusion}\n\n"

        # Save the guide
        with open("output/complete_guide.md", "w") as f:
            f.write(guide_content)

        print("\nComplete guide compiled and saved to output/complete_guide.md")
        return "Guide creation completed successfully"

def kickoff():
    """Run the guide creator flow"""
    GuideCreatorFlow().kickoff()
    print("\n=== Flow Complete ===")
    print("Your comprehensive guide is ready in the output directory.")
    print("Open output/complete_guide.md to view it.")

def plot():
    """Generate a visualization of the flow"""
    flow = GuideCreatorFlow()
    flow.plot("guide_creator_flow")
    print("Flow visualization saved to guide_creator_flow.html")

if __name__ == "__main__":
    kickoff()
```


---

# 1) Import e contesto

```python
import json, os
from typing import List, Dict
from pydantic import BaseModel, Field
from crewai import LLM
from crewai.flow.flow import Flow, listen, start
from guide_creator_flow.crews.content_crew.content_crew import ContentCrew
```

* **Pydantic** serve per definire **modelli tipizzati** (schemi) che validano e normalizzano i dati.
* **LLM** è il wrapper di CrewAI per chiamare il modello.
* **Flow**, **@start**, **@listen** sono il mini-framework per definire **pipeline dichiarative** a step.

---

# 2) Modelli Pydantic (schema dei dati)

```python
class Section(BaseModel):
    title: str = Field(description="Title of the section")
    description: str = Field(description="Brief description of what the section should cover")

class GuideOutline(BaseModel):
    title: str = Field(description="Title of the guide")
    introduction: str = Field(description="Introduction to the topic")
    target_audience: str = Field(description="Description of the target audience")
    sections: List[Section] = Field(description="List of sections in the guide")
    conclusion: str = Field(description="Conclusion or summary of the guide")
```

* Ogni classe eredita da **BaseModel**: questo abilita **validazione**, **conversione tipi**, e metodi di parsing/serializzazione (es. `model_dump()`, `model_dump_json()` in Pydantic v2; `dict()`, `json()` in v1).
* `Field(description=...)` aggiunge **metadati** (utile anche per documentazione/LLM structured output).
* `sections: List[Section]` annida un’altra classe: Pydantic valida **ricorsivamente**.

### Stato del flow (Pydantic come “single source of truth”)

```python
class GuideCreatorState(BaseModel):
    topic: str = ""
    audience_level: str = ""
    guide_outline: GuideOutline = None
    sections_content: Dict[str, str] = {}
```

* È l’oggetto che **vive tra gli step** del flow (sarà `self.state`).
* **Nota importante:** meglio scrivere

  ```python
  from typing import Optional
  guide_outline: Optional[GuideOutline] = None
  ```

  così il tipo riflette che all’inizio è assente.
* `sections_content: Dict[str, str] = {}`: con Pydantic **va bene** (Pydantic crea copie per evitare “mutable default shared”). Con dataclass standard servirebbe `default_factory=dict`.

---

# 3) Cos’è il Flow e come funziona

```python
class GuideCreatorFlow(Flow[GuideCreatorState]):
    ...
```

* Estende `Flow` e **parametrizza** lo stato (`Flow[GuideCreatorState]`): CrewAI saprà che `self.state` ha quel tipo.
* Gli step del flow sono **metodi** decorati:

  * `@start()`: **primo** step.
  * `@listen(<step_precedente>)`: esegui **dopo** lo step indicato, ricevendone in input il valore di ritorno.

Il flow finale ha **3 step in catena**:

1. `get_user_input` → raccoglie input e popola `self.state`.
2. `create_guide_outline` → chiama l’LLM, valida il JSON in `GuideOutline`, salva su file.
3. `write_and_compile_guide` → per ogni sezione genera contenuto con una sub-crew, compila il Markdown finale e lo salva.

---

# 4) Step 1 – Input utente e popolamento stato

```python
@start()
def get_user_input(self):
    self.state.topic = input("...")
    # loop di validazione sul livello
    ...
    return self.state
```

* L’**input** viene messo dentro lo **stato** (topic e audience\_level).
* Il `return self.state` consente allo step successivo di ricevere **lo stato aggiornato**.

---

# 5) Step 2 – Chiamata LLM, JSON e Pydantic

```python
@listen(get_user_input)
def create_guide_outline(self, state):
    llm = LLM(model="azure/gpt-4o", response_format=GuideOutline)
    messages = [
      {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
      {"role": "user", "content": f"""Create a detailed outline ... """}
    ]
    response = llm.call(messages=messages)

    outline_dict = json.loads(response)
    self.state.guide_outline = GuideOutline(**outline_dict)
```

### Cosa succede qui, con precisione:

1. **Prompt** a messaggi: chiedi al modello di restituire **JSON** (testo nel formato giusto).

2. `LLM(..., response_format=GuideOutline)`: *intenzione* di ottenere output strutturato. A seconda della versione/provider, questo può:

   * **A) Non avere effetto** (il modello risponde testo JSON “puro”) → usi `json.loads`.
   * **B) Supportare structured output** → potresti ricevere già un JSON conforme.

   Visto che **poi usi `json.loads(response)`**, il codice assume che `response` sia una **stringa JSON**.

3. **Parsing JSON → oggetto Python**: `outline_dict = json.loads(response)`.

4. **Validazione con Pydantic**: `GuideOutline(**outline_dict)`:

   * Pydantic **verifica** che ci siano tutti i campi (title, introduction, target\_audience, sections\[], conclusion) e che `sections` contenga oggetti coerenti con `Section`.
   * Se qualcosa **non torna**, alza un `ValidationError` (ottimo per fail-fast).

5. **Persistenza su file**:

   ```python
   os.makedirs("output", exist_ok=True)
   with open("output/guide_outline.json","w") as f:
       json.dump(outline_dict, f, indent=2)
   ```

   * Salvi il **JSON originale** (non l’oggetto Pydantic). Utile per audit/debug.

**Suggerimento Pydantic (più robusto e pulito):**

* Se usi **Pydantic v2**, puoi validare direttamente la stringa JSON:

  ```python
  self.state.guide_outline = GuideOutline.model_validate_json(response)
  ```
* Se usi **Pydantic v1**:

  ```python
  self.state.guide_outline = GuideOutline.parse_raw(response)
  ```

Così **eviti** il passaggio manuale con `json.loads` e l’`**` unpack.

---

# 6) Step 3 – Generare le sezioni e compilare il Markdown

```python
@listen(create_guide_outline)
def write_and_compile_guide(self, outline):
    completed_sections = []
    for section in outline.sections:
        # contesto dei precedenti
        previous_sections_text = ...
        result = ContentCrew().crew().kickoff(inputs={
            "section_title": section.title,
            "section_description": section.description,
            "audience_level": self.state.audience_level,
            "previous_sections": previous_sections_text,
            "draft_content": ""
        })
        self.state.sections_content[section.title] = result.raw
        completed_sections.append(section.title)
```

* `outline` è il **valore ritornato** dallo step precedente (qui è l’oggetto `GuideOutline`).
* Per ogni sezione:

  * Costruisci un **contesto** con il testo delle sezioni già scritte (utile per coerenza).
  * Lanci una **sub-crew** (`ContentCrew`) che scrive la sezione (ritorna testo in `result.raw`).
  * Salvi nel **dizionario** `sections_content[title] = content`.

### Compilazione e salvataggio:

```python
guide_content = f"# {outline.title}\n\n"
guide_content += f"## Introduction\n\n{outline.introduction}\n\n"
for section in outline.sections:
    guide_content += f"\n\n{self.state.sections_content.get(section.title, '')}\n\n"
guide_content += f"## Conclusion\n\n{outline.conclusion}\n\n"

with open("output/complete_guide.md", "w") as f:
    f.write(guide_content)
```

* Si crea un **Markdown** concatenando introduzione, sezioni (nell’ordine definito da `outline.sections`) e conclusione.
* Si salva su disco in `output/complete_guide.md`.

---

# 7) Avvio e visualizzazione del flow

```python
def kickoff():
    GuideCreatorFlow().kickoff()
    print("...")

def plot():
    flow = GuideCreatorFlow()
    flow.plot("guide_creator_flow")
```

* `kickoff()` costruisce l’istanza del flow e **lo esegue**, facendo scorrere gli step in ordine (start → listen → listen).
* `plot(...)` genera una **visualizzazione HTML** della pipeline per capire in un colpo d’occhio gli step e le dipendenze.

---

## Riepilogo specifico su **Pydantic** e **JSON**

* **Pydantic** ti garantisce che la struttura del dato prodotto dall’LLM rispetti lo **schema**: è un “contratto” tra prompt e codice.
* **Pipeline tipica**:

  1. LLM restituisce **stringa JSON** → 2) **parse** (meglio `model_validate_json` / `parse_raw`) → 3) ottieni un **oggetto tipizzato** (`GuideOutline`) → 4) usi l’oggetto in sicurezza nel resto del flow → 5) (opzionale) **salvi** JSON su file per tracciabilità.
* **Vantaggi**: se l’LLM sbaglia formato o manca un campo, lo sai **subito** (ValidationError), non quando stai già generando file.

---

## Migliorie consigliate (piccole, ma utili)

1. **Optional corretto per guide\_outline**

```python
from typing import Optional
guide_outline: Optional[GuideOutline] = None
```

2. **Parsing diretto con Pydantic**

* v2:

  ```python
  self.state.guide_outline = GuideOutline.model_validate_json(response)
  ```
* v1:

  ```python
  self.state.guide_outline = GuideOutline.parse_raw(response)
  ```

3. **Gestione errori/robustezza** (utile con provider diversi)

```python
try:
    self.state.guide_outline = GuideOutline.model_validate_json(response)
except Exception as e:
    print("Outline non valido:", e)
    # fallback: provare a ripulire il testo o rilanciare la richiesta all’LLM
    raise
```

4. **Consistenza tra prompt e schema**

* Il prompt chiede: titolo, introduzione, target\_audience, 4-6 sezioni, conclusione.
* Lo **schema** `GuideOutline` richiede esattamente questi campi: ottimo. Mantieni i nomi coerenti nel prompt per aumentare la probabilità di JSON valido.



# Creare agenti in CrewAI

## 1) Due modalità possibili

* **YAML (consigliata):** definisci gli agenti in file di configurazione esterni. È più pulito e facile da mantenere.
* **Direttamente in codice:** crei gli agenti con Python. Utile per prototipi veloci, ma meno ordinato nel tempo.

---

## 2) Configurazione via YAML (consigliata)

### Dove si trova e cosa modificare

Dopo aver creato il progetto CrewAI, apri:

```
src/latest_ai_development/config/agents.yaml
```

e personalizza il template secondo le tue esigenze.

### Sostituzione delle variabili

Nei file YAML puoi usare segnaposto come `{topic}`. Quando avvii la crew, passi i valori tramite `inputs` e CrewAI li sostituisce automaticamente:

```python
crew.kickoff(inputs={'topic': 'AI Agents'})
```

### Esempio di `agents.yaml`

```yaml
# src/latest_ai_development/config/agents.yaml
researcher:
  role: >
    {topic} Senior Data Researcher
  goal: >
    Uncover cutting-edge developments in {topic}
  backstory: >
    You're a seasoned researcher with a knack for uncovering the latest
    developments in {topic}. Known for your ability to find the most relevant
    information and present it in a clear and concise manner.

reporting_analyst:
  role: >
    {topic} Reporting Analyst
  goal: >
    Create detailed reports based on {topic} data analysis and research findings
  backstory: >
    You're a meticulous analyst with a keen eye for detail. You're known for
    your ability to turn complex data into clear and concise reports, making
    it easy for others to understand and act on the information you provide.
```

**Cosa contiene:**

* **Chiavi dell’agente** (`researcher`, `reporting_analyst`): sono i “nomi” degli agenti.
* **`role`**: il ruolo con il segnaposto `{topic}`.
* **`goal`**: l’obiettivo dell’agente, ancora parametrico su `{topic}`.
* **`backstory`**: il contesto narrativo che guida lo stile/approccio dell’agente.

---

## 3) Usare il YAML nel codice

Crea una classe crew che eredita da `CrewBase` e punta al file YAML. Poi definisci un metodo per ciascun agente.

```python
# src/latest_ai_development/crew.py
from crewai import Agent, Crew, Process
from crewai.project import CrewBase, agent, crew
from crewai_tools import SerperDevTool

@CrewBase
class LatestAiDevelopmentCrew():
  """LatestAiDevelopment crew"""

  # 1) Indichi dove si trova la configurazione degli agenti
  agents_config = "config/agents.yaml"

  # 2) Definisci un metodo per ogni agente presente nel YAML

  @agent
  def researcher(self) -> Agent:
    return Agent(
      # Carica dal YAML la sezione 'researcher'
      config=self.agents_config['researcher'],  # type: ignore[index]
      verbose=True,
      # Eventuali tool dell'agente (qui: SerperDevTool)
      tools=[SerperDevTool()]
    )

  @agent
  def reporting_analyst(self) -> Agent:
    return Agent(
      # Carica dal YAML la sezione 'reporting_analyst'
      config=self.agents_config['reporting_analyst'],  # type: ignore[index]
      verbose=True
    )
```

**Punti chiave:**

* `agents_config` indica a CrewAI dove leggere le definizioni YAML.
* Ogni metodo decorato con `@agent` **deve** restituire un `Agent` costruito a partire dalla rispettiva sezione YAML tramite `config=...`.
* Puoi impostare `verbose=True` per avere output più dettagliati.
* Puoi associare strumenti all’agente tramite `tools=[...]` (nell’esempio viene passato `SerperDevTool()`).

---

## 4) Corrispondenza dei nomi

I **nomi** usati nel file YAML (es. `researcher`, `reporting_analyst`) **devono corrispondere** ai **nomi dei metodi** nella classe Python (`def researcher(self)`, `def reporting_analyst(self)`).
Questa corrispondenza è ciò che permette a CrewAI di collegare la configurazione alla definizione del singolo agente nel codice.

---

## 5) Avvio con input

Quando lanci la crew, passa gli input che valorizzano i segnaposto:

```python
crew.kickoff(inputs={'topic': 'AI Agents'})
```

Così `{topic}` verrà sostituito ovunque compaia nelle definizioni degli agenti.


---

# Gestione e creazione dei **Tasks** in CrewAI

## 1) Cos’è un Task

In CrewAI un **Task** rappresenta un incarico specifico assegnato a un **Agent**.
Contiene tutte le informazioni necessarie per eseguire il lavoro: descrizione, obiettivo, agente responsabile, strumenti da utilizzare, eventuali output attesi e altre proprietà.

I task possono:

* essere **individuali** (un singolo agente li esegue),
* oppure **collaborativi**, con più agenti che cooperano. In questo caso la logica di collaborazione è gestita dal **process** della crew.

---

## 2) Esecuzione dei Task

I task possono seguire due modalità di esecuzione:

1. **Sequential (sequenziale):**
   Vengono eseguiti nell’ordine in cui sono definiti.

2. **Hierarchical (gerarchico):**
   L’assegnazione è basata sui ruoli e sulle competenze degli agenti.

Esempio di definizione della crew con i task:

```python
crew = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    process=Process.sequential  # oppure Process.hierarchical
)
```

---

## 3) Attributi di un Task

Un Task ha diverse proprietà configurabili. Le principali sono:

| Attributo                         | Tipo             | Descrizione                                                       |
| --------------------------------- | ---------------- | ----------------------------------------------------------------- |
| **description**                   | `str`            | Testo che descrive cosa deve fare il task.                        |
| **expected\_output**              | `str`            | Descrizione di come appare un completamento corretto del task.    |
| **name** (opzionale)              | `str`            | Identificativo del task.                                          |
| **agent** (opzionale)             | `BaseAgent`      | Agente incaricato di svolgerlo.                                   |
| **tools** (opzionale)             | `List[BaseTool]` | Strumenti che l’agente può usare per questo task.                 |
| **context** (opzionale)           | `List[Task]`     | Altri task le cui uscite diventano input per questo task.         |
| **async\_execution** (opzionale)  | `bool`           | Se eseguire in modo asincrono. Default: `False`.                  |
| **human\_input** (opzionale)      | `bool`           | Se richiedere revisione umana dell’output. Default: `False`.      |
| **markdown** (opzionale)          | `bool`           | Se l’output deve essere formattato in Markdown. Default: `False`. |
| **config** (opzionale)            | `dict`           | Parametri di configurazione specifici del task.                   |
| **output\_file** (opzionale)      | `str`            | Percorso file dove salvare l’output.                              |
| **create\_directory** (opzionale) | `bool`           | Se creare la directory di output se non esiste. Default: `True`.  |
| **output\_json** (opzionale)      | `BaseModel`      | Modello Pydantic per strutturare output JSON.                     |
| **output\_pydantic** (opzionale)  | `BaseModel`      | Modello Pydantic per tipizzare l’output.                          |
| **callback** (opzionale)          | `Any`            | Funzione o oggetto da eseguire al termine del task.               |
| **guardrail** (opzionale)         | `Callable`       | Funzione che valida l’output prima di passare al task successivo. |

---

## 4) Creazione dei Task

Anche qui esistono **due modalità**:

* tramite **YAML** (consigliato),
* direttamente in **codice Python**.

---

### A) YAML Configuration (consigliata)

È il metodo più pulito e facilmente gestibile.
Dopo aver creato il progetto, vai in:

```
src/latest_ai_development/config/tasks.yaml
```

e personalizza le definizioni.

Quando avvii la crew, i placeholder (es. `{topic}`) saranno sostituiti con i valori forniti in `inputs`.

Esempio:

```python
crew.kickoff(inputs={'topic': 'AI Agents'})
```

#### Esempio di `tasks.yaml`

````yaml
research_task:
  description: >
    Conduct a thorough research about {topic}
    Make sure you find any interesting and relevant information given
    the current year is 2025.
  expected_output: >
    A list with 10 bullet points of the most relevant information about {topic}
  agent: researcher

reporting_task:
  description: >
    Review the context you got and expand each topic into a full section for a report.
    Make sure the report is detailed and contains any and all relevant information.
  expected_output: >
    A fully fledge report with the main topics, each with a full section of information.
    Formatted as markdown without '```'
  agent: reporting_analyst
  markdown: true
  output_file: report.md
````

---

### B) Utilizzare il YAML nel codice

Si definisce una classe crew che eredita da `CrewBase`.
Dentro la classe si mappano:

* agenti (da `agents.yaml`),
* task (da `tasks.yaml`),
* e il processo di esecuzione.

```python
# src/latest_ai_development/crew.py

from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool

@CrewBase
class LatestAiDevelopmentCrew():
  """LatestAiDevelopment crew"""

  # Definizione agenti
  @agent
  def researcher(self) -> Agent:
    return Agent(
      config=self.agents_config['researcher'],  # type: ignore[index]
      verbose=True,
      tools=[SerperDevTool()]
    )

  @agent
  def reporting_analyst(self) -> Agent:
    return Agent(
      config=self.agents_config['reporting_analyst'],  # type: ignore[index]
      verbose=True
    )

  # Definizione task
  @task
  def research_task(self) -> Task:
    return Task(
      config=self.tasks_config['research_task']  # type: ignore[index]
    )

  @task
  def reporting_task(self) -> Task:
    return Task(
      config=self.tasks_config['reporting_task']  # type: ignore[index]
    )

  # Crew con agenti, task e process
  @crew
  def crew(self) -> Crew:
    return Crew(
      agents=[
        self.researcher(),
        self.reporting_analyst()
      ],
      tasks=[
        self.research_task(),
        self.reporting_task()
      ],
      process=Process.sequential
    )
```

---

## 5) Punti chiave da ricordare

* I **task** sono assegnazioni precise per un agente.
* Possono essere eseguiti in modo **sequenziale** o **gerarchico**.
* Ogni task ha attributi configurabili (descrizione, output atteso, agente, ecc.).
* **Consigliato**: definire i task in **YAML** e poi importarli nel codice.
* I **nomi dei task nel YAML** devono corrispondere ai **metodi in Python** (`research_task`, `reporting_task`).

---




# Crews in CrewAI

## 1) Cos’è una **Crew**

In CrewAI, una **crew** è un gruppo collaborativo di **agents** che lavorano insieme per completare una serie di **tasks**.
La crew definisce:

* la **strategia** di esecuzione dei task,
* le **modalità di collaborazione** tra agenti,
* il **workflow complessivo** del progetto.

---

## 2) Attributi principali di una Crew

Una crew ha diversi attributi che ne definiscono comportamento e configurazione.

| Attributo                              | Parametri     | Descrizione                                                                                              |
| -------------------------------------- | ------------- | -------------------------------------------------------------------------------------------------------- |
| **tasks**                              | `List[Task]`  | Elenco dei task da eseguire.                                                                             |
| **agents**                             | `List[Agent]` | Agenti che fanno parte della crew.                                                                       |
| **process** (opzionale)                | `Process`     | Definisce il flusso di esecuzione (`sequential` o `hierarchical`). Default: `sequential`.                |
| **verbose** (opzionale)                | `bool`        | Livello di dettaglio nei log. Default: `False`.                                                          |
| **manager\_llm** (opzionale)           | `LLM`         | Modello linguistico usato dal manager agent nei processi gerarchici. Obbligatorio se usi `hierarchical`. |
| **function\_calling\_llm** (opzionale) | `LLM`         | LLM per il function calling sugli strumenti. Ogni agente può comunque sovrascriverlo.                    |
| **config** (opzionale)                 | `dict`        | Configurazioni extra della crew in formato JSON o dizionario.                                            |
| **max\_rpm** (opzionale)               | `int`         | Numero massimo di richieste al minuto. Utile per rispettare limiti di rate.                              |
| **memory** (opzionale)                 | `object`      | Memoria di esecuzione: breve termine, lungo termine o entità.                                            |
| **cache** (opzionale)                  | `bool`        | Se salvare i risultati degli strumenti in cache. Default: `True`.                                        |
| **embedder** (opzionale)               | `dict`        | Configurazione dell’embedder (default: `{"provider": "openai"}`). Usato principalmente con la memoria.   |
| **step\_callback** (opzionale)         | `Callable`    | Funzione eseguita dopo ogni step di ogni agente. Utile per logging personalizzato.                       |
| **task\_callback** (opzionale)         | `Callable`    | Funzione eseguita al termine di ogni task. Utile per monitoraggio o operazioni aggiuntive.               |
| **share\_crew** (opzionale)            | `bool`        | Se condividere con il team CrewAI informazioni di esecuzione per migliorare la libreria.                 |
| **output\_log\_file** (opzionale)      | `str`         | Percorso file in cui salvare i log (`.txt` o `.json`).                                                   |
| **manager\_agent** (opzionale)         | `Agent`       | Imposta un agente personalizzato come manager.                                                           |
| **prompt\_file** (opzionale)           | `str`         | File JSON contenente il prompt da usare per la crew.                                                     |
| **planning** (opzionale)               | `bool`        | Se attivato, un *AgentPlanner* analizza i dati della crew e genera un piano da allegare a ogni task.     |
| **planning\_llm** (opzionale)          | `LLM`         | LLM usato dall’AgentPlanner per la fase di planning.                                                     |
| **knowledge\_sources** (opzionale)     | `list`        | Fonti di conoscenza condivise a livello di crew, accessibili a tutti gli agenti.                         |

### Nota importante

* Se imposti **`max_rpm`** sulla crew, questo valore sovrascrive i limiti definiti a livello di singolo agente.

---

## 3) Creazione delle Crew

Esistono due modi:

1. **via YAML** (consigliato, in coerenza con agents e tasks),
2. direttamente in **codice Python**.

---

## 4) Esempio di Crew con decorators

Un esempio tipico è definire una classe che eredita da `CrewBase` e usare i decorator per marcare agenti, task e crew.

```python
from crewai import Agent, Crew, Task, Process
from crewai.project import CrewBase, agent, task, crew, before_kickoff, after_kickoff
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List

@CrewBase
class YourCrewName:
    """Description of your crew"""

    agents: List[BaseAgent]
    tasks: List[Task]

    # Percorsi ai file YAML
    agents_config = 'config/agents.yaml'
    tasks_config = 'config/tasks.yaml'

    @before_kickoff
    def prepare_inputs(self, inputs):
        # Eseguito prima dell’avvio della crew
        inputs['additional_data'] = "Some extra information"
        return inputs

    @after_kickoff
    def process_output(self, output):
        # Eseguito dopo la conclusione della crew
        output.raw += "\nProcessed after kickoff."
        return output

    @agent
    def agent_one(self) -> Agent:
        return Agent(
            config=self.agents_config['agent_one'],  # type: ignore[index]
            verbose=True
        )

    @agent
    def agent_two(self) -> Agent:
        return Agent(
            config=self.agents_config['agent_two'],  # type: ignore[index]
            verbose=True
        )

    @task
    def task_one(self) -> Task:
        return Task(
            config=self.tasks_config['task_one']  # type: ignore[index]
        )

    @task
    def task_two(self) -> Task:
        return Task(
            config=self.tasks_config['task_two']  # type: ignore[index]
        )

    @crew
    def crew(self) -> Crew:
        return Crew(
            agents=self.agents,  # Raccolti automaticamente dai decorator @agent
            tasks=self.tasks,    # Raccolti automaticamente dai decorator @task
            process=Process.sequential,
            verbose=True,
        )
```

---

## 5) Esecuzione della crew

Per avviare la crew:

```python
YourCrewName().crew().kickoff(inputs={"any": "input here"})
```

I task saranno eseguiti **nell’ordine definito**, perché il processo è sequenziale.

---

## 6) Decorators disponibili

CrewAI mette a disposizione vari decorator che semplificano la gestione:

* **@CrewBase** → indica la classe come base per una crew.
* **@agent** → metodo che restituisce un `Agent`.
* **@task** → metodo che restituisce un `Task`.
* **@crew** → metodo che restituisce la `Crew`.
* **@before\_kickoff** → metodo eseguito *prima* dell’avvio della crew.
* **@after\_kickoff** → metodo eseguito *dopo* la conclusione della crew.

Questi decorator permettono di organizzare meglio la struttura e di evitare di dover elencare manualmente agenti e task.

---





# Crew Output in CrewAI

## 1) Cos’è il CrewOutput

Quando una **crew** viene eseguita, il risultato non è un semplice testo, ma un oggetto strutturato: la classe **`CrewOutput`**.
Questa classe permette di accedere ai risultati in diversi formati (testo grezzo, JSON, Pydantic) e di analizzare anche i dettagli di esecuzione, come l’uso dei token o l’output dei singoli task.

---

## 2) Attributi principali di CrewOutput

| Attributo         | Tipo                       | Descrizione                                                              |
| ----------------- | -------------------------- | ------------------------------------------------------------------------ |
| **raw**           | `str`                      | Output grezzo della crew (formato base di default).                      |
| **pydantic**      | `Optional[BaseModel]`      | Oggetto Pydantic con output strutturato (se definito).                   |
| **json\_dict**    | `Optional[Dict[str, Any]]` | Output in formato JSON come dizionario.                                  |
| **tasks\_output** | `List[TaskOutput]`         | Lista con gli output dei singoli task eseguiti.                          |
| **token\_usage**  | `Dict[str, Any]`           | Riepilogo dei token consumati, utile per monitorare prestazioni e costi. |

---

## 3) Metodi e proprietà utili

* **`.json`** → restituisce l’output in formato JSON (stringa).
* **`.to_dict()`** → converte JSON e Pydantic in un dizionario.
* **`__str__`** → restituisce una rappresentazione testuale dell’output, dando priorità a:

  1. Pydantic,
  2. JSON,
  3. Raw.

---

## 4) Accesso ai risultati dopo l’esecuzione

Dopo aver lanciato una crew con `.kickoff()`, puoi accedere all’output:

```python
# Esempio di esecuzione
crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, write_article_task],
    verbose=True
)

crew_output = crew.kickoff()

# Accesso all’output
print(f"Raw Output: {crew_output.raw}")
if crew_output.json_dict:
    print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")
if crew_output.pydantic:
    print(f"Pydantic Output: {crew_output.pydantic}")
print(f"Tasks Output: {crew_output.tasks_output}")
print(f"Token Usage: {crew_output.token_usage}")
```

---

## 5) Gestione dei log

Puoi salvare un log completo dell’esecuzione con l’attributo `output_log_file`:

```python
# Salvataggio log
crew = Crew(output_log_file=True)           # -> logs.txt
crew = Crew(output_log_file="mylog.txt")    # -> mylog.txt
crew = Crew(output_log_file="mylog.json")   # -> mylog.json (in JSON)
```

Se `output_log_file` è `False` o `None`, i log non vengono salvati.

---

## 6) Memoria e Cache

* **Memory**: una crew può avere memoria a breve termine, lungo termine o per entità. Questo consente di ricordare esperienze passate e migliorare le decisioni future.
* **Cache**: permette di salvare i risultati degli strumenti, così da non ripetere task identici e velocizzare le esecuzioni.

---

## 7) Metriche di utilizzo

Dopo l’esecuzione, puoi accedere a metriche generali di utilizzo LLM tramite `usage_metrics`:

```python
crew = Crew(agents=[agent1, agent2], tasks=[task1, task2])
crew.kickoff()
print(crew.usage_metrics)
```

---

## 8) Flusso di esecuzione

* **Sequential** → i task vengono eseguiti in ordine, uno dopo l’altro.
* **Hierarchical** → un agente manager coordina gli altri, delegando compiti e validando risultati.

  > In questo caso è obbligatorio definire `manager_llm` o `manager_agent`.

---

## 9) Avvio della crew

Il metodo `.kickoff()` dà il via al workflow e restituisce un **CrewOutput**:

```python
# Avvio della crew
result = my_crew.kickoff()
print(result)  # Stampa l’output finale
```

---





# Flows in CrewAI

## 1) Cos’è un Flow

Un **Flow** in CrewAI è un modo strutturato per costruire **workflow AI multi-step**.
Ti permette di:

* combinare ed eseguire più **tasks** e **crews**,
* condividere **stato** tra i vari passaggi,
* applicare **logica condizionale** (if, loop, branching),
* avere un’architettura **event-driven**, cioè basata su eventi e reazioni.

In altre parole, i **Flows** servono a orchestrare processi complessi, dove l’output di un passaggio diventa l’input del successivo.

---

## 2) Caratteristiche principali

* **Workflow semplificati** → puoi concatenare task e crew per creare processi complessi.
* **Gestione dello stato** → i dati prodotti in un passaggio restano disponibili negli step successivi.
* **Architettura event-driven** → i metodi possono “ascoltare” eventi di altri metodi.
* **Controllo flessibile** → puoi introdurre condizioni, cicli e rami alternativi.

---

## 3) Primo esempio: Generare una città e un fun fact

Ecco un esempio di **Flow** con due step:

```python
from crewai.flow.flow import Flow, listen, start
from dotenv import load_dotenv
from litellm import completion

class ExampleFlow(Flow):
    model = "gpt-4o-mini"

    @start()
    def generate_city(self):
        print("Starting flow")
        print(f"Flow State ID: {self.state['id']}")  # ID univoco della run

        response = completion(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": "Return the name of a random city in the world.",
                },
            ],
        )

        random_city = response["choices"][0]["message"]["content"]

        # Salvo la città nello stato del flow
        self.state["city"] = random_city
        print(f"Random City: {random_city}")
        return random_city

    @listen(generate_city)
    def generate_fun_fact(self, random_city):
        response = completion(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": f"Tell me a fun fact about {random_city}",
                },
            ],
        )

        fun_fact = response["choices"][0]["message"]["content"]
        self.state["fun_fact"] = fun_fact
        return fun_fact

# Avvio del flow
flow = ExampleFlow()
flow.plot()      # Mostra il diagramma
result = flow.kickoff()

print(f"Generated fun fact: {result}")
```

### Cosa succede:

1. Ogni esecuzione del flow riceve un **ID univoco (UUID)**.
2. Il metodo `generate_city()` parte come step iniziale e restituisce una città casuale.
3. Lo stato (`self.state`) viene aggiornato con la città.
4. Il metodo `generate_fun_fact()` “ascolta” l’output di `generate_city` e lo usa per generare un fatto curioso.
5. Entrambi i risultati restano salvati nello **stato globale del flow**.

---

## 4) Decorators principali

I decorator sono ciò che definisce la logica del Flow.

### `@start()`

* Indica che il metodo è un **punto di avvio**.
* Puoi avere più metodi `@start()`: verranno eseguiti in parallelo.
* Quando il Flow parte, tutti gli step marcati `@start()` vengono lanciati.

### `@listen()`

* Indica che un metodo deve **ascoltare** l’output di un altro metodo.
* Si attiva solo quando lo step che sta ascoltando termina.
* Riceve in input l’output del metodo collegato.

#### Uso 1 – ascoltare un metodo per nome:

```python
@listen("generate_city")
def generate_fun_fact(self, random_city):
    # Implementazione
```

#### Uso 2 – ascoltare un metodo direttamente:

```python
@listen(generate_city)
def generate_fun_fact(self, random_city):
    # Implementazione
```

---

## 5) Stato del Flow

* Ogni Flow ha un **dizionario `self.state`** che contiene:

  * `id` → l’UUID univoco dell’esecuzione,
  * variabili salvate nei passaggi (`city`, `fun_fact`, ecc.).

Questa memoria interna consente di mantenere il **contesto** lungo tutta la catena di esecuzione.

---

## 6) Requisiti

Perché funzioni l’esempio:

* Devi avere un file `.env` con la variabile `OPENAI_API_KEY`.
* Serve per autenticare le richieste verso OpenAI (o il provider LLM che scegli).

---



# Flow Control in CrewAI

I **Flows** non si limitano a concatenare metodi in sequenza: permettono di gestire la logica di esecuzione con condizioni, routing e ascolto di più eventi.
Vediamo i tre meccanismi principali: **or**, **and** e **router**.

---

## 1) Conditional Logic con `or_`

Con `or_` puoi far sì che un metodo venga eseguito se **almeno uno** tra più metodi emette un output.

### Esempio:

```python
from crewai.flow.flow import Flow, listen, or_, start

class OrExampleFlow(Flow):

    @start()
    def start_method(self):
        return "Hello from the start method"

    @listen(start_method)
    def second_method(self):
        return "Hello from the second method"

    @listen(or_(start_method, second_method))
    def logger(self, result):
        print(f"Logger: {result}")

flow = OrExampleFlow()
flow.plot("my_flow_plot")
flow.kickoff()
```

### Cosa succede:

* `start_method` viene eseguito per primo.
* `second_method` ascolta il suo output e parte dopo.
* `logger` ascolta sia `start_method` sia `second_method`.
* Quindi, appena **uno dei due** produce un risultato, `logger` si attiva e lo stampa.

---

## 2) Conditional Logic con `and_`

Con `and_` invece il metodo listener si attiva **solo quando tutti i metodi specificati hanno emesso un output**.

### Esempio:

```python
from crewai.flow.flow import Flow, and_, listen, start

class AndExampleFlow(Flow):

    @start()
    def start_method(self):
        self.state["greeting"] = "Hello from the start method"

    @listen(start_method)
    def second_method(self):
        self.state["joke"] = "What do computers eat? Microchips."

    @listen(and_(start_method, second_method))
    def logger(self):
        print("---- Logger ----")
        print(self.state)

flow = AndExampleFlow()
flow.plot()
flow.kickoff()
```

### Cosa succede:

* `start_method` salva un saluto nello stato.
* `second_method`, che ascolta il primo, aggiunge una battuta.
* Solo quando **entrambi** hanno prodotto output, `logger` si attiva e stampa lo stato.

---

## 3) Conditional Routing con `@router()`

Il decorator `@router()` serve per definire **percorsi condizionali** in base all’output di un metodo.
In pratica, a seconda del valore restituito, il flusso prende strade diverse.

### Esempio:

```python
import random
from crewai.flow.flow import Flow, listen, router, start
from pydantic import BaseModel

class ExampleState(BaseModel):
    success_flag: bool = False

class RouterFlow(Flow[ExampleState]):

    @start()
    def start_method(self):
        print("Starting the structured flow")
        random_boolean = random.choice([True, False])
        self.state.success_flag = random_boolean

    @router(start_method)
    def second_method(self):
        if self.state.success_flag:
            return "success"
        else:
            return "failed"

    @listen("success")
    def third_method(self):
        print("Third method running")

    @listen("failed")
    def fourth_method(self):
        print("Fourth method running")

flow = RouterFlow()
flow.plot("my_flow_plot")
flow.kickoff()
```

### Cosa succede:

1. `start_method` genera casualmente un booleano (`True` o `False`).
2. `second_method` con `@router` decide il percorso:

   * se `True`, restituisce `"success"`,
   * se `False`, restituisce `"failed"`.
3. `third_method` ascolta il percorso `"success"`,
   `fourth_method` ascolta `"failed"`.
4. A seconda del valore casuale, verrà eseguito **solo uno dei due**.

---

## 4) In sintesi

* **`or_`** → esegue il listener quando **uno qualsiasi** dei metodi ascoltati produce output.
* **`and_`** → esegue il listener quando **tutti** i metodi ascoltati producono output.
* **`@router`** → devia il flusso su percorsi diversi in base al valore restituito.

---




# Tools in CrewAI

## 1) Che cos’è un Tool

In CrewAI, un **Tool** è una **funzione/abilità** che un agente può utilizzare per svolgere azioni specifiche.
Può trattarsi di:

* strumenti di ricerca sul web,
* analisi di dati,
* lettura di file e directory,
* interazione con API esterne,
* supporto alla collaborazione tra agenti.

In breve: i **tools estendono le capacità degli agenti** permettendo loro di eseguire compiti che altrimenti non sarebbero possibili.

---

## 2) Caratteristiche principali dei Tools

* **Utility** → creati per attività concrete come ricerche web, analisi di documenti o generazione di contenuti.
* **Integrazione** → si collegano facilmente al workflow degli agenti, arricchendone le possibilità.
* **Customizzabilità** → puoi usare tool predefiniti oppure crearne di nuovi, adatti al tuo progetto.
* **Error Handling** → includono meccanismi di gestione degli errori, così il flusso non si interrompe.
* **Caching** → evitano di ripetere operazioni identiche salvando i risultati già ottenuti.
* **Asynchronous Support** → supportano anche esecuzioni asincrone, per non bloccare l’agente.

---

## 3) Installazione dei Tools

Per usare i tool extra forniti da CrewAI:

```bash
pip install 'crewai[tools]'
```

---

## 4) Esempio pratico: integrare i tools in una crew

Ecco un esempio completo che mostra come utilizzare vari strumenti.

```python
import os
from crewai import Agent, Task, Crew
# Importiamo i tools disponibili
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Imposta le API keys (necessarie per i tool esterni)
os.environ["SERPER_API_KEY"] = "Your Key"  # chiave per serper.dev
os.environ["OPENAI_API_KEY"] = "Your Key"

# 1) Istanziare i tools
docs_tool = DirectoryReadTool(directory='./blog-posts')  # legge una cartella
file_tool = FileReadTool()                              # legge file singoli
search_tool = SerperDevTool()                           # ricerca web con serper.dev
web_rag_tool = WebsiteSearchTool()                      # ricerca e rag su siti web

# 2) Creare gli agenti con i tools
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],   # assegno i tool di ricerca
    verbose=True
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],       # assegno i tool di lettura file
    verbose=True
)

# 3) Definire i tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description="Write an engaging blog post about the AI industry, based on the research analyst's summary. Draw inspiration from the latest blog posts in the directory.",
    expected_output="A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.",
    agent=writer,
    output_file='blog-posts/new_post.md'  # il risultato sarà salvato qui
)

# 4) Assemblare la crew con planning abilitato
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True  # abilita la pianificazione
)

# 5) Avviare l’esecuzione
crew.kickoff()
```

---

## 5) Repository Enterprise

Nella versione **CrewAI Enterprise** c’è anche un **Tools Repository** che include:

* connettori già pronti per sistemi aziendali comuni,
* interfaccia per creare tool personalizzati,
* versioning e condivisione dei tool,
* sicurezza e conformità integrate.

---

## 6) Benefici operativi

* Tutti i tools integrano **gestione errori** → gli agenti possono continuare anche in caso di problemi.
* Tutti i tools supportano **caching intelligente** → i risultati già ottenuti vengono riutilizzati.
* Puoi controllare il caching con l’attributo `cache_function` di ciascun tool.

---



# Creare e utilizzare Tools personalizzati in CrewAI

## 1) Gestione automatica delle chiamate

Il framework **CrewAI** gestisce in automatico sia i **tools sincroni** che quelli **asincroni**.
 Questo significa che non serve preoccuparsi se un tool è `sync` o `async`: l’agente potrà chiamarlo nello stesso modo.

---

## 2) Creare un tool con il decorator `@tool`

Per definire un tuo tool personalizzato, si usa il decorator `@tool` fornito da CrewAI.

### Esempio:

```python
from crewai.tools import tool

@tool("Name of my tool")
def my_tool(question: str) -> str:
    """Clear description for what this tool is useful for, your agent will need this information to use it."""
    # Qui va la logica del tool
    return "Result from your custom tool"
```

---

## 3) Struttura del tool

* **`@tool("Name of my tool")`** → assegna un nome leggibile al tool.
  Questo è ciò che l’agente userà per identificarlo.

* **Docstring (""" ... """)** → descrive in modo chiaro a cosa serve il tool.
  La descrizione è fondamentale, perché gli agenti la usano per capire **quando e come invocare** il tool.

* **Funzione Python** → contiene la logica operativa. Può essere semplice (una query, un calcolo, una trasformazione) o complessa (chiamate API, elaborazioni di file, ecc.).

* **Valore di ritorno** → l’output che l’agente riceverà quando utilizza il tool.

---

## 4) Come viene usato dagli agenti

Una volta creato, puoi passare il tool a uno o più agenti nel parametro `tools=[...]`:

```python
my_custom_tool = my_tool

agent = Agent(
    role="Assistant",
    goal="Answer specific questions using a custom tool",
    backstory="You can query external knowledge using a tool.",
    tools=[my_custom_tool],   # qui aggiungo il tool
    verbose=True
)
```

---

## 5) Punti chiave da ricordare

* Non serve distinguere manualmente tra tools sincroni e asincroni.
* Il nome dato nel decorator deve essere chiaro e rappresentativo.
* La **docstring** è fondamentale: più è chiara, più l’agente saprà quando usare correttamente il tool.
* Puoi creare quanti tool personalizzati vuoi e combinarli con quelli già forniti da CrewAI.

---

---

# Guardrails per i Tool in CrewAI

## 1) Cos’è un “guardrail”

Un **guardrail** è un controllo che **impone limiti e regole**:

* **prima** dell’uso del tool (validazione input),
* **durante** l’esecuzione (gestione errori),
* **dopo** l’uso (validazione output).

Nel framework:

* a livello **Task** esiste l’attributo **`guardrail`** (una funzione) che **valida l’output del task** prima di passare allo step successivo.
* a livello **Tool** conviene inserire **controlli interni** (input/output + docstring chiara) così l’agente lo usa nel modo giusto.

---

## 2) Regole base per i tool (semplici e efficaci)

1. **Docstring chiara = primo guardrail**
   Spiega cosa fa il tool, **cosa non fa**, formati accettati, limiti. L’agente decide *quando* usarlo proprio leggendo questa descrizione.

2. **Type hints + controlli input**
   Usa annotazioni di tipo e valida parametri (liste di allow/deny, lunghezze massime, pattern ammessi).

3. **Gestione errori**
   Cattura eccezioni e restituisci messaggi utili (o alza errori mirati) invece di far “saltare” il flusso.

4. **Validazione output**
   Verifica che l’output abbia **forma/struttura** attesa (es. almeno N bullet, JSON parsabile, campi non vuoti).

5. **(Opzionale, lato Task)** `guardrail=`
   Aggiungi una funzione di verifica finale nel **Task** per bloccare output non conformi **prima** di passare al prossimo step.

---

## 3) Esempio minimo: tool con guardrail “interno”

```python
from crewai.tools import tool

ALLOWED_TOPICS = {"ai", "ml", "nlp"}

@tool("Topic-limited search")
def topic_search(query: str) -> str:
    """
    Cerca informazioni SOLO su questi topic: ai, ml, nlp.
    Input: una query breve (<= 120 caratteri).
    Output: testo riassuntivo in 3-5 frasi.
    """
    # Guardrail INPUT
    if not isinstance(query, str) or len(query.strip()) == 0:
        raise ValueError("Query vuota o non valida.")
    if len(query) > 120:
        raise ValueError("Query troppo lunga (max 120 caratteri).")

    topic = query.split()[0].lower()
    if topic not in ALLOWED_TOPICS:
        raise ValueError(f"Topic non ammesso: {topic}")

    # ... qui la tua logica (API, ricerca, ecc.) con try/except ...
    summary = "Breve riassunto (3-5 frasi) sull'argomento richiesto."

    # Guardrail OUTPUT
    if summary.count(".") < 3:  # controllo grezzo per 3+ frasi
        raise ValueError("Output troppo breve: servono almeno 3 frasi.")
    return summary
```

**Cosa fa bene:**

* limita i **topic**,
* impone una **lunghezza** massima della query,
* garantisce **forma minima** dell’output.

---

## 4) Esempio: guardrail lato **Task** (validazione finale)

Usa l’attributo `guardrail` del Task per verificare che il risultato di **tutto il task** sia conforme.

```python
from crewai import Agent, Task, Crew

def report_guardrail(output: str) -> bool:
    """
    Accetta solo output Markdown con:
    - almeno 3 bullet ('- ' o '* ')
    - titolo in prima riga (inizia con '# ')
    """
    if not isinstance(output, str):
        return False
    lines = output.strip().splitlines()
    if not lines or not lines[0].startswith("# "):
        return False
    bullets = sum(1 for ln in lines if ln.lstrip().startswith(("- ", "* ")))
    return bullets >= 3

writer = Agent(
    role="Report Writer",
    goal="Scrivere report Markdown coerenti",
    backstory="Autore tecnico attento agli standard.",
)

write_task = Task(
    description="Scrivi un report Markdown con titolo e almeno 3 bullet.",
    expected_output="Markdown ben formattato, titolo H1 e >=3 bullet.",
    agent=writer,
    markdown=True,
    guardrail=report_guardrail,  # <<< Guardrail lato Task
)

crew = Crew(agents=[writer], tasks=[write_task], verbose=True)
crew.kickoff()
```

**Cosa succede:**

* il task può usare qualsiasi tool,
* **prima di chiudere**, il `guardrail` controlla l’output: se non passa, il flusso **non prosegue**.

---

## 5) Schema “applico i guardrail dove servono”

* **Tool (IN):** docstring + validate input (tipi, lunghezze, allowlist).
* **Tool (RUN):** try/except, messaggi chiari, fallback.
* **Tool (OUT):** verifica formato/completezza (es. JSON valido, conteggio frasi/bullet).
* **Task (FINAL):** `guardrail=` per bloccare output non conformi dell’intero step.
* **(Se serve)** `human_input=True` nel Task per una revisione umana finale.

---

## 6) Mini check-list operativa

* [ ] La **docstring** del tool dice chiaramente cosa fa e cosa non fa.
* [ ] Gli **input** sono validati (tipo, lunghezza, allow/deny).
* [ ] Gli **errori** sono gestiti (niente stacktrace all’utente).
* [ ] L’**output** rispetta un formato minimo (o un modello Pydantic/JSON).
* [ ] (Task) Ho messo un **`guardrail`** finale se il formato del risultato è cruciale.

---



# Esercizio 1 — “Parafrasa le keyword”

### Obiettivo

Far eseguire a un **Flow** una **Crew** con **1 agent** che usa un tool di keyword e un LLM per parafrasare in 1 frase.

### Cosa usa

* Flow
* Crew (1 agent)
* Tool semplice
* LLM nell’agent

### Consegna

Crea un tool che restituisce **3 keyword**.
L’agent deve chiamare il tool e generare **una sola frase** che le integra in modo naturale.

---

### Codice

```python
# --- Tool semplice ---
@tool("Keywords Tool")
def get_keywords(_: str) -> str:
    """Ritorna 3 keyword di esempio da inserire in una frase."""
    return "AI, machine learning, data analysis"



# --- Task: deve usare il tool e produrre 1 sola frase ---
task = Task(
    description=(
        "Chiama 'Keywords Tool' per ottenere 3 keyword e scrivi UNA SOLA frase "
        "che le integra naturalmente (massimo 25 parole)."
    ),
    expected_output="Una frase unica con le 3 keyword integrate.",
    agent=summarizer,
    markdown=False,
)
```





# Esercizio 2 — “Scelta del Tool: ricerca o calcolo”

### Obiettivo

Creare un **agent interattivo** che chieda all’utente quale operazione eseguire:

1. Fare una **ricerca web** (usando i tool integrati di CrewAI).
2. Calcolare la **somma di due numeri** (con un tool definito da te).

L’agent deve quindi:

* chiedere la scelta all’utente,
* chiedere gli input corretti a seconda del tool selezionato,
* chiamare il tool corrispondente,
* restituire il risultato finale.

---

### Suggerimenti

* Per la **ricerca web** puoi utilizzare i **tool integrati** di CrewAI (es. `SerperDevTool`).
* Per la **somma**, crea un tuo tool personalizzato.
* Ricorda: CrewAI passa automaticamente all’AI la descrizione del tool (docstring + firma della funzione).

  > Questo significa che l’AI cercherà da sola di inserire i parametri giusti quando deve richiamarlo.

---

### Consegna

1. Definisci un tool `add_numbers(a: int, b: int)` che ritorna la somma.
2. Configura un agent che faccia domande all’utente:

   * “Vuoi fare una ricerca web o una somma?”
   * Se ricerca → chiedi l’argomento.
   * Se somma → chiedi due numeri.
3. L’agent esegue il tool corrispondente e restituisce il risultato in una frase chiara.

---

### Codice del tool di somma

```python
from crewai.tools import tool

# --- Tool somma ---
@tool("Add Numbers Tool")
def add_numbers(a: int, b: int) -> int:
    """Calcola la somma di due numeri interi e ritorna il risultato."""
    return a + b
```

---

# Esercizio 3 — “Tool RAG a tema (domain-scoped)”

## Obiettivo

Creare un **tool RAG** che contiene documenti **solo su un argomento specifico** (es. *“Spring Boot Security”*).
Un **agent** deve:

1. capire se la domanda dell’utente riguarda *quell’argomento*,
2. **usare il tool RAG** se sì,
3. altrimenti rispondere senza tool (o dire che non ha fonti pertinenti).

---

## Perché è utile

* Evita “allucinazioni” fuori dominio.
* Garantisce risposte **tracciabili** (con passaggi e citazioni).
* Allena l’agent alla **scelta del tool** in base al contesto (routing).

---

## Suggerimenti & Tips per studenti

* **Definite bene il dominio**: una frase chiara nel docstring (es. “Questo tool copre SOLO Spring Boot Security”).
* **Piccolo corpus, alta qualità**: 5–15 passaggi densi, ognuno con titolo, fonte, testo.
* **Output strutturato**: sempre “answer + fonti”.
* **Fallback on-topic**: se la domanda è vaga ma su tema vicino, chiedi **chiarimenti** prima di usare il tool.
* **Off-topic = rifiuta con gentilezza**: spiega che il tool è limitato e proponi alternative.
* **Prompt del tool**: guida l’LLM a citare e non inventare.
* **Valuta**: conserva *query, passaggi scelti, risposta* per revisione (RAGAS/valutazione manuale).
* **Debias**: preferisci passaggi recenti o “ufficiali” quando possibile; evita mixare voci non affidabili.
* **Stopword & sinonimi**: se implementi un retriever semplice, aggiungi dizionari sinonimi/lemmatizzazione.
* **Sicurezza**: impedisci che l’LLM risponda fuori ambito “come se sapesse”: obbliga sempre a **citare**.

---


# Esercizio 4 — “Unit conversion (km↔m) con validazione”

**Obiettivo:** creare un tool semplicissimo di conversione e farlo chiamare all’agent.
**Cosa usare:** 1 agent, 1 tool `convert_units`, 1 task.
**Consegna:**

* Supporta `km_to_m(value)` e `m_to_km(value)`.
* L’agent deve chiedere direzione e valore; se mancano → chiede chiarimento.

```python
from crewai.tools import tool

@tool("Unit Converter")
def convert_units(direction: str, value: float) -> float:
    """direction in {'km_to_m', 'm_to_km'}; converte e ritorna il valore."""
    if direction == "km_to_m":
        return value * 1000.0
    if direction == "m_to_km":
        return value / 1000.0
    return float("nan")  # semplice segnale d'errore
```